 # First raw analysis 

In [1]:
%run 30_testing_different_bandwidths.ipynb

data_folder = ../Psychopy_data/pilot/B_theta_B_sf
Overwriting tmp/30_testing_different_bandwidths.py
]0;IPython: Notebooks/W17
 Initializing
B_thetas =  [0.09817477 0.18005332 0.33021925 0.60562477 1.11072073]
B_sfs =  [0.0625 0.125  0.25   0.5    1.    ]


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os

In [3]:
subj_name_list =  ['LP', 'JF']
subj_name_list =  ['LP', ]

subj_data_list = []
import json
for subject in subj_name_list :
    x = json.load(open(os.path.join(data_folder, 'Psychophys_discrim_%s.json' % subject), 'rb'))
     #changer et mettre le path du fichier où se trouve les psychophysic data
    subj_data_list.append(x)

In [4]:

B_theta_list = []
for x in subj_data_list[0] :
    B_theta_list.append(x[3]*180/np.pi)
B_theta_list = sorted(set(B_theta_list))

print('B_theta_list=', B_theta_list)
B_sf_list = []
for x in subj_data_list[0] :
    B_sf_list.append(x[4])
B_sf_list = sorted(set(B_sf_list))
print('B_sf_list=', B_sf_list)

B_theta_list= [5.625, 10.316295486052553, 18.920169343208578, 34.69974357167867, 63.63961030678928]
B_sf_list= [0.0625, 0.125, 0.25, 0.5, 1.0]


## B_sf/ B_theta

In [5]:
data_dict = {}

for num, data in enumerate(subj_data_list) :
    data_dict[num] = {}
    N_data = len(data)
    data_dict[num]['theta'] = np.zeros((1, N_data))
    data_dict[num]['conditions'] = np.zeros((1, N_data)).astype(np.int)
    data_dict[num]['response'] = np.zeros((1, N_data))
    for i, item in enumerate(data):
        #print(i, item)
        data_dict[num]['theta'][0, i] = item[1]*180/np.pi
        data_dict[num]['conditions'][0, i] = item[5]+item[6]*len(B_theta_list)
        data_dict[num]['response'][0, i] = 1 if item[2] == 'right' else 0

In [6]:
import torch
from torch.utils.data import TensorDataset, DataLoader

torch.set_default_tensor_type("torch.DoubleTensor")
criterion = torch.nn.BCELoss(reduction="sum")

N_conditions = len(B_sf_list)*len(B_theta_list)
bias = True
logit0_init, theta0_init, log_wt_init = -np.log(1/.10 - 1), 0.0, np.log(4.)

class LogisticRegressionModel(torch.nn.Module):
    def __init__(self, logit0_init, theta0_init, log_wt_init, bias=True):
        super(LogisticRegressionModel, self).__init__()
        if bias:
            self.theta0 = torch.nn.Parameter(theta0_init)
        else:
            self.theta0 = theta0_init
        self.logit0 = torch.nn.Parameter(logit0_init)
        self.log_wt = torch.nn.Parameter(log_wt_init)
        self.do_indep = self.logit0.shape.numel() > 1
        
    def forward(self, theta, i_condition):
        if self.do_indep:
            p0 = torch.sigmoid(self.logit0[i_condition.to(int)])
        else:
            p0 = torch.sigmoid(self.logit0)
            
        theta0 = self.theta0[i_condition.to(int)]
        wt = torch.exp(self.log_wt[i_condition.to(int)])
        out = p0 / 2 + (1 - p0) * torch.sigmoid((theta-theta0)/wt)
        return out

learning_rate = 0.005
beta1, beta2 = 0.9, 0.999
betas = (beta1, beta2)
num_epochs = 2 ** 12 + 1

def fit_data(logistic_model,
    theta, ind_condition, y,
    learning_rate=learning_rate,
    # batch_size=batch_size,  # previous notebook showed that learning_rate had no influence on performance
    num_epochs=num_epochs,
    betas=betas,
    verbose=False, **kwargs
):

    Theta, Ind_condition, labels = torch.Tensor(theta[:, None]), torch.ByteTensor(ind_condition[:, None]), torch.Tensor(y[:, None])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    Theta, Ind_condition, labels = Theta.to(device), Ind_condition.to(device), labels.to(device)        
    
    logistic_model = logistic_model.to(device)
    logistic_model.train()
    optimizer = torch.optim.Adam(logistic_model.parameters(), lr=learning_rate, betas=betas)
            
    for epoch in range(int(num_epochs)):
        outputs = logistic_model(Theta, Ind_condition)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if verbose and (epoch % (num_epochs // 32) == 0):
            print(f"Iteration: {epoch} - Loss: {loss.item() /len(theta):.5f}")
            losses = []

    logistic_model.eval()
    Theta, Ind_condition, labels = torch.Tensor(theta[:, None]), torch.Tensor(ind_condition[:, None]), torch.Tensor(y[:, None])
    outputs = logistic_model(Theta, Ind_condition)
    loss = criterion(outputs, labels).item() / len(theta)
    logistic_model.cpu()
    return logistic_model, loss

In [7]:
bias = False
#bias = True
logit0_init = logit0_init * torch.ones(1)
# indep = logit0_init=logit0_init * torch.ones(int(len(data)/len(B_theta_list)/len(B_sf_list)))
results = {}
for num, data in enumerate(subj_data_list) :
    logistic_model = LogisticRegressionModel(logit0_init * torch.ones(1), theta0_init*torch.ones(N_conditions), torch.log(log_wt_init*torch.ones(N_conditions)), bias=bias)
    logistic_model, loss = fit_data(logistic_model, data_dict[num]['theta'], 
                                    data_dict[num]['conditions'], 
                                    data_dict[num]['response'], verbose=True)
    results[num] = logistic_model
    print("Final loss =", loss)

Iteration: 0 - Loss: 316.55038
Iteration: 128 - Loss: 269.11273
Iteration: 256 - Loss: 242.51238
Iteration: 384 - Loss: 231.57964
Iteration: 512 - Loss: 227.50920
Iteration: 640 - Loss: 225.73848
Iteration: 768 - Loss: 224.87052
Iteration: 896 - Loss: 224.42057
Iteration: 1024 - Loss: 224.17352
Iteration: 1152 - Loss: 224.03040
Iteration: 1280 - Loss: 223.94324
Iteration: 1408 - Loss: 223.88658
Iteration: 1536 - Loss: 223.84263
Iteration: 1664 - Loss: 223.59224
Iteration: 1792 - Loss: 223.52987
Iteration: 1920 - Loss: 223.50709
Iteration: 2048 - Loss: 223.49432
Iteration: 2176 - Loss: 223.48545
Iteration: 2304 - Loss: 223.47879
Iteration: 2432 - Loss: 223.47363
Iteration: 2560 - Loss: 223.46955
Iteration: 2688 - Loss: 223.46629
Iteration: 2816 - Loss: 223.46365
Iteration: 2944 - Loss: 223.46147
Iteration: 3072 - Loss: 223.45967
Iteration: 3200 - Loss: 223.45815
Iteration: 3328 - Loss: 223.45687
Iteration: 3456 - Loss: 223.45578
Iteration: 3584 - Loss: 223.45484
Iteration: 3712 - Loss: 

In [8]:
for num in range(2): print(results[num].logit0)

Parameter containing:
tensor([-1.8153], requires_grad=True)


KeyError: 1

In [ ]:
def npa2str(npa):
    """
    format a numpy array into a string
    """
    return ', '.join(list(map('{:.3f}'.format, npa)))

def get_params(logistic_model, verbose=False):
    theta0_ = logistic_model.theta0.detach().numpy()
    wt_ = torch.exp(logistic_model.log_wt).detach().numpy()
    p0_ = torch.sigmoid(logistic_model.logit0).detach().numpy()
    N_conditions = wt_.shape[0]
    if verbose:
        #for i_condition in range(N_conditions):
        #    print(f'-> i_condition={i_condition}')
        #    if bias:
        #        print(f"theta0 = {theta0_[i_condition]:.3f}")
        #    print(f"slope = {wt_[i_condition]:.3f}")
        if bias:
            print(f"theta0 = {npa2str(theta0_)}")
        print(f"slope = {npa2str(wt_)}")
        print(f"p0 = {npa2str(p0_)}")
    return theta0_, wt_, p0_

for num, data in enumerate(subj_data_list) :
    print('--> observer num', num)
    theta0_, wt_, p0_ = get_params(results[num], verbose=True)

In [ ]:
fig, axs = plt.subplots(len(B_sf_list), len(subj_data_list), figsize=(15, 21))
for num, data in enumerate(subj_data_list) :
    print('--> observer num', num)
    x_values = np.linspace(-25, 25, 400)

    for i_B_sf, B_sf in enumerate(B_sf_list):
        if len(subj_data_list)>1:
            ax = axs[i_B_sf][num]
        else:
            ax = axs[i_B_sf]

        #colors = plt.cm.inferno(np.linspace(1, .2, len(B_thetas))) #tc colormap

        for i_B_theta, B_theta in enumerate(B_theta_list):
            i_condition = i_B_theta + i_B_sf*len(B_theta_list)
            conditions = i_condition*np.ones_like(x_values)
            y_values = results[num](torch.Tensor(x_values), 
                                      torch.ByteTensor(conditions)).detach().numpy()
            ax.plot(x_values, y_values, label=r'$B_\theta=%s$' % f'{B_theta:.1f}')

        ax.set_xlabel(r"orientation $\theta$ (°)", fontsize=12)
        ax.set_yticks([0.0, 1.0])
        ax.set_yticklabels(["Left", "Right"], fontsize=12)
        ax.legend(fontsize=11, frameon=False, scatterpoints=6)
        if num==0:  ax.legend(fontsize=11, frameon=False, scatterpoints=6)
        ax.text(14, 0.1, f'B_sf = {B_sf:.3f}', fontsize=12)
        

In [ ]:
fig, axs = plt.subplots(1, len(subj_data_list), figsize=(8,6)) 
for num, data in enumerate(subj_data_list) :
    print('--> observer num', num)
    theta0_, wt_, p0_ = get_params(results[num], verbose=True)
    coef_list = []
    for i_B_sf, B_sf in enumerate(B_sf_list):
        wt__ = wt_[(i_B_sf*len(B_theta_list)):((i_B_sf+1)*len(B_theta_list))]
        axs[num].plot(B_theta_list, np.log(wt__), label=f'B_sf = {B_sf:.3f}')
    axs[num].set_xlabel(r' $B_\theta$ (°)')
    axs[num].set_ylabel(r' log slopes ')
    axs[num].legend(loc ='best') 
    axs[num].set_xscale('log')
    